In [1]:
import os
import csv
import sys
module_path = os.path.abspath(os.path.join('C:\\Users\\koolk\\Desktop\\brain-diffusion\\Chad_functions_and_unittests'))
if module_path not in sys.path:
    sys.path.append(module_path)

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scipy.optimize as opt
import scipy.stats as stat
from mpl_toolkits.mplot3d import Axes3D
from operator import itemgetter
import random
import numpy as np
import numpy.ma as ma
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D

pi = np.pi
sin = np.sin
cos = np.cos

In [31]:
#The purpose of this book is to compare in one plot the results of varying parameters.

nameRED_cor = '100nm_PEG_in_105_S2_cortex'
nameYG_cor = '60nm_PEG_in_105_S2_cortex'

nameRED_cor2 = '100nm_PS_PEG_in_cortex'

geoRED_cor = np.genfromtxt('./RED/geoM2xy_{}.csv'.format(nameRED_cor), delimiter =",")
geoYG_cor = np.genfromtxt('./YG/geoM2xy_{}.csv'.format(nameYG_cor), delimiter =",")
geoRED_cor2 = np.genfromtxt('./RED/geoM2xy_{}.csv'.format(nameRED_cor2), delimiter =",")

In [29]:
conversion = dict()

conversion[1] = (0.16, 4.98, 1)

frames = 99
frames_1 = np.linspace(0, frames, frames+1).astype(np.int64)
time = frames_1/conversion[1][1]
time2 = frames_1/2

array([  0. ,   0.5,   1. ,   1.5,   2. ,   2.5,   3. ,   3.5,   4. ,
         4.5,   5. ,   5.5,   6. ,   6.5,   7. ,   7.5,   8. ,   8.5,
         9. ,   9.5,  10. ,  10.5,  11. ,  11.5,  12. ,  12.5,  13. ,
        13.5,  14. ,  14.5,  15. ,  15.5,  16. ,  16.5,  17. ,  17.5,
        18. ,  18.5,  19. ,  19.5,  20. ,  20.5,  21. ,  21.5,  22. ,
        22.5,  23. ,  23.5,  24. ,  24.5,  25. ,  25.5,  26. ,  26.5,
        27. ,  27.5,  28. ,  28.5,  29. ,  29.5,  30. ,  30.5,  31. ,
        31.5,  32. ,  32.5,  33. ,  33.5,  34. ,  34.5,  35. ,  35.5,
        36. ,  36.5,  37. ,  37.5,  38. ,  38.5,  39. ,  39.5,  40. ,
        40.5,  41. ,  41.5,  42. ,  42.5,  43. ,  43.5,  44. ,  44.5,
        45. ,  45.5,  46. ,  46.5,  47. ,  47.5,  48. ,  48.5,  49. ,  49.5])

In [23]:
#Figure parameters

filename = 'geo_cor.png'

frame_m = 14

limit1=3.1 #x range
tick1 = 1

limit2=24.1 #y range
tick2 = 4 #y ticks

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=0 #decimals displayed on x axis
dec2=0 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [7]:
def color_variant(hex_color, brightness_offset=1):
    """ takes a color like #87c95f and produces a lighter or darker variant """
    if len(hex_color) != 7:
        raise Exception("Passed %s into color_variant(), needs to be in #87c95f format." % hex_color)
    rgb_hex = [hex_color[x:x+2] for x in [1, 3, 5]]
    new_rgb_int = [int(hex_value, 16) + brightness_offset for hex_value in rgb_hex]
    new_rgb_int = [min([255, max([0, i])]) for i in new_rgb_int] # make sure new values are between 0 and 255
    # hex() produces "0x88", we want just "88"
    return "#" + "".join([hex(i)[2:] for i in new_rgb_int])

In [8]:
# for name, hex in mpl.colors.cnames.items():
#     print(name, hex)

# g2 = color_variant('#008000', brightness_offset=0)
# b2 = color_variant('#0000FF', brightness_offset=0)

In [9]:
def clamp(val, minimum=0, maximum=255):
    if val < minimum:
        return minimum
    if val > maximum:
        return maximum
    return val

def colorscale(hexstr, scalefactor):
    """
    Scales a hex string by ``scalefactor``. Returns scaled hex string.

    To darken the color, use a float value between 0 and 1.
    To brighten the color, use a float value greater than 1.

    >>> colorscale("#DF3C3C", .5)
    #6F1E1E
    >>> colorscale("#52D24F", 1.6)
    #83FF7E
    >>> colorscale("#4F75D2", 1)
    #4F75D2
    """

    hexstr = hexstr.strip('#')

    if scalefactor < 0 or len(hexstr) != 6:
        return hexstr

    r, g, b = int(hexstr[:2], 16), int(hexstr[2:4], 16), int(hexstr[4:], 16)

    r = round(clamp(r * scalefactor))
    g = round(clamp(g * scalefactor))
    b = round(clamp(b * scalefactor))

    return "#%02x%02x%02x" % (r, g, b)

In [10]:
g2 = colorscale('#008000', 1)
b2 = colorscale('#0000FF', 10)
r2 = colorscale('#FF0000', 10)
g2

'#008000'

In [11]:
time[0:-1].shape

(99,)

In [25]:
# Creates figure
fig = plt.figure(figsize=(18, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time[0:frame_m], geoRED_cor[0:frame_m], 'g--', linewidth=wi, dashes=[d1,d2], label='100nm PEG')
ax.plot(time[0:frame_m], geoYG_cor[0:frame_m], 'b--', linewidth=wi, dashes=[d1,d2], label='60nm YG')

# ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(time1[1:], line1[0][1:], linewidth=wi-3, color=b2)
# ax.plot(time1[1:], line1[1][1:], linewidth=wi-3, color=g2)
# ax.plot(time1[1:], line1[2][1:], linewidth=wi-3, color=r2)
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.02, 0.80), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# plt.yscale('log')
# plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0.00001, limit2])

# Save your figure
plt.savefig('{}'.format(filename), bbox_inches='tight')
plt.show()

In [28]:
#Figure parameters

filename = 'geo_comp_het_WT.png'

frame_m = 14

limit1=3.1 #x range
tick1 = 1

limit2=24.1 #y range
tick2 = 4 #y ticks

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=0 #decimals displayed on x axis
dec2=0 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [35]:
# Creates figure
fig = plt.figure(figsize=(18, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time[0:frame_m], geoRED_cor[0:frame_m], 'g--', linewidth=wi, dashes=[d1,d2], label='Het')
ax.plot(time2[0:7], geoRED_cor2[0:7], 'b--', linewidth=wi, dashes=[d1,d2], label='WT')

# ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(time1[1:], line1[0][1:], linewidth=wi-3, color=b2)
# ax.plot(time1[1:], line1[1][1:], linewidth=wi-3, color=g2)
# ax.plot(time1[1:], line1[2][1:], linewidth=wi-3, color=r2)
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.02, 0.80), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# plt.yscale('log')
# plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0.00001, limit2])

# Save your figure
plt.savefig('{}'.format(filename), bbox_inches='tight')
plt.show()